# lvr parser

### Download files

http://plvr.land.moi.gov.tw/DownloadOpenData

In [36]:
from functools import reduce
import pandas as pd
import requests
from pathlib import Path
from clint.textui import progress
import zipfile
import os
import re

In [7]:
def download_file(url, path):
    r = requests.head(url)
    filename = r.headers.get('Content-Disposition').split('=')[-1].replace('"', '')
    total_length = int(r.headers.get('content-length'))
    
    local_filename = Path(path).expanduser() / filename
    
    print(f'Saving file to "{local_filename}..."')
    # NOTE the stream=True parameter
    r = requests.get(url, stream=True)
    with open(local_filename, 'wb') as f:
        for chunk in progress.bar(r.iter_content(chunk_size=1024), expected_size=(total_length/1024) + 1): 
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
                f.flush() # commented by recommendation from J.F.Sebastian
    return True

In [8]:
url_base = 'http://plvr.land.moi.gov.tw/DownloadHistory?type=season&fileName={year}S{season}'

want_data = [('105', '1'), ('105', '2'), ('105', '3'), ('105', '4'), ('106', '1'), ('106', '2'), ('106', '3')]
urls = [url_base.format(year=d[0], season=d[1]) for d in want_data]
urls

['http://plvr.land.moi.gov.tw/DownloadHistory?type=season&fileName=105S1',
 'http://plvr.land.moi.gov.tw/DownloadHistory?type=season&fileName=105S2',
 'http://plvr.land.moi.gov.tw/DownloadHistory?type=season&fileName=105S3',
 'http://plvr.land.moi.gov.tw/DownloadHistory?type=season&fileName=105S4',
 'http://plvr.land.moi.gov.tw/DownloadHistory?type=season&fileName=106S1',
 'http://plvr.land.moi.gov.tw/DownloadHistory?type=season&fileName=106S2',
 'http://plvr.land.moi.gov.tw/DownloadHistory?type=season&fileName=106S3']

In [9]:
# for url in urls:
#     download_file(url, './data')

Saving file to "data/2016S1.zip..."
Saving file to "data/2016S2.zip..."
Saving file to "data/2016S3.zip..."
Saving file to "data/2016S4.zip..."
Saving file to "data/2017S1.zip..."
Saving file to "data/2017S2.zip..."
Saving file to "data/2017S3.zip..."


### Unzip

In [46]:
zip_files = list(Path('./data').glob('*S*.zip'))
zip_files

[PosixPath('data/2017S1.zip'),
 PosixPath('data/2016S1.zip'),
 PosixPath('data/2017S3.zip'),
 PosixPath('data/2016S3.zip'),
 PosixPath('data/2016S2.zip'),
 PosixPath('data/2016S4.zip'),
 PosixPath('data/2017S2.zip')]

In [14]:
p = zip_files[0]
dest_folder = p.parent / p.stem

In [20]:
!cd data; ls

2016S1.zip  2016S4.zip	2017S2.zip  df_all_2017S3.csv
2016S2.zip  2017S1	2017S3	    df_all_2017S3_geocode.csv
2016S3.zip  2017S1.zip	2017S3.zip  dt_2017S3_geo_codebase.csv


In [25]:
overwrite = True

if dest_folder.exists() and overwrite:
    dest_folder.mkdir(exist_ok=True)
    
    with zipfile.ZipFile(p.as_posix(), 'r') as zip_ref:
        print('Unzip file {}...'.format(p.as_posix()))
        zip_ref.extractall(dest_folder.as_posix())
        print('Finished')
elif dest_folder.exists() and not overwrite:
    raise FileExistsError(dest_folder.as_posix())

Unzip file data/2017S1.zip...
Finished


In [51]:
def unzip_lvr(zip_files, overwrite=True):
    for p in zip_files:
        p = Path(p)
        dest_folder = p.parent / p.stem; print(dest_folder)
        
        if not dest_folder.exists() or overwrite:
            dest_folder.mkdir(exist_ok=True)

            with zipfile.ZipFile(p.as_posix(), 'r') as zip_ref:
                print('Unzip file {}...'.format(p.as_posix()))
                zip_ref.extractall(dest_folder.as_posix())
                print('Finished')
        elif dest_folder.exists() and not overwrite:
            raise FileExistsError(dest_folder.as_posix())
            
    return True

In [52]:
unzip_lvr(zip_files)

data/2017S1
Unzip file data/2017S1.zip...
Finished
data/2016S1
Unzip file data/2016S1.zip...
Finished
data/2017S3
Unzip file data/2017S3.zip...
Finished
data/2016S3
Unzip file data/2016S3.zip...
Finished
data/2016S2
Unzip file data/2016S2.zip...
Finished
data/2016S4
Unzip file data/2016S4.zip...
Finished
data/2017S2
Unzip file data/2017S2.zip...
Finished


True

---

### List csv files

In [5]:
print(os.listdir(DEST_FOLDER)[:10], '\n\n-----------------\n')
dest_folder = Path(DEST_FOLDER)

csv_files = [f.as_posix() for f in dest_folder.glob('*.CSV') if re.search(r'land_[AB].CSV$', f.as_posix())]
csv_files.sort()

print('\n'.join(csv_files))

['F_lvr_land_A.XML', 'I_lvr_land_C_LAND.XML', 'A_lvr_land_A_LAND.TXT', 'Q_lvr_land_A_LAND.CSV', 'O_lvr_land_C_LAND.CSV', 'J_lvr_land_B_LAND.XML', 'J_lvr_land_C.XML', 'U_lvr_land_C.CSV', 'C_lvr_land_A_LAND.XML', 'M_lvr_land_A_BUILD.XML'] 

-----------------

data/2017S3/A_lvr_land_A.CSV
data/2017S3/A_lvr_land_B.CSV
data/2017S3/B_lvr_land_A.CSV
data/2017S3/B_lvr_land_B.CSV
data/2017S3/C_lvr_land_A.CSV
data/2017S3/C_lvr_land_B.CSV
data/2017S3/D_lvr_land_A.CSV
data/2017S3/D_lvr_land_B.CSV
data/2017S3/E_lvr_land_A.CSV
data/2017S3/E_lvr_land_B.CSV
data/2017S3/F_lvr_land_A.CSV
data/2017S3/F_lvr_land_B.CSV
data/2017S3/G_lvr_land_A.CSV
data/2017S3/G_lvr_land_B.CSV
data/2017S3/H_lvr_land_A.CSV
data/2017S3/H_lvr_land_B.CSV
data/2017S3/I_lvr_land_A.CSV
data/2017S3/I_lvr_land_B.CSV
data/2017S3/J_lvr_land_A.CSV
data/2017S3/J_lvr_land_B.CSV
data/2017S3/K_lvr_land_A.CSV
data/2017S3/K_lvr_land_B.CSV
data/2017S3/M_lvr_land_A.CSV
data/2017S3/M_lvr_land_B.CSV
data/2017S3/N_lvr_land_A.CSV
data/2017S3/N_lvr

---

### Parse data

In [18]:
import pandas as pd

In [19]:
# !cd data/2017S3/ && iconv_ext.sh cp950 utf-8

def pd_read_csv_cp950(file):
    try:
        df = pd.read_csv(file, encoding='cp950')
    except Exception as e:
        from io import StringIO
        with open(file , "rb") as  fin :
            text = fin.read()
            text = text.decode('cp950', errors='ignore')
            df = pd.read_csv(StringIO(text))
#         print(f, '---', e)
    return df

In [23]:
def pd_read_lrv(file):
    
    county_dict = dict(
        [('C', '基隆市'),
        ('A', '臺北市'),
        ('F', '新北市'),
        ('H', '桃園縣'),
        ('O', '新竹市'),
        ('J', '新竹縣'),
        ('K', '苗栗縣'),
        ('B', '臺中市'),
        ('M', '南投縣'),
        ('N', '彰化縣'),
        ('P', '雲林縣'),
        ('I', '嘉義市'),
        ('Q', '嘉義縣'),
        ('D', '臺南市'),
        ('E', '高雄市'),
        ('T', '屏東縣'),
        ('G', '宜蘭縣'),
        ('U', '花蓮縣'),
        ('V', '臺東縣'),
        ('X', '澎湖縣'),
        ('W', '金門縣'),
        ('Z', '連江縣')]
    )
    county_code = file.split('/')[-1][0]
    county = county_dict.get(county_code)
    df = pd_read_csv_cp950(file)
    df.insert(loc=0, column='county', value=county)
    return df

In [24]:
pd_read_lrv('data/2017S3/B_lvr_land_C.CSV')

,county,鄉鎮市區,租賃標的,土地區段位置或建物區門牌,租賃總面積平方公尺,都市土地使用分區,非都市土地使用分區,非都市土地使用編定,租賃年月日,租賃筆棟數,...,建物現況格局-隔間,有無管理組織,有無附傢俱,總額元,單價每平方公尺,車位類別,租賃總面積平方公尺.2,租金總額元,備註,編號
0,臺中市,西屯區,房地(土地+建物),臺中市西屯區長安路二段91~120號,95.00,商,NaN,NaN,1050528,土地1建物1車位0,...,有,無,有,15000,185.0,NaN,0.00,0,NaN,RPPPMLLJNHMFFCB67CC
1,臺中市,西屯區,房地(土地+建物),臺中市西屯區長安路二段91~120號,95.00,住,NaN,NaN,1060428,土地1建物1車位0,...,有,無,有,11000,94.0,NaN,0.00,0,NaN,RPQPMLLJNHMFFCB77CC
2,臺中市,西屯區,房地(土地+建物),臺中市西屯區西安街151~180號,74.00,住,NaN,NaN,1060414,土地1建物1車位0,...,無,無,無,63000,472.0,NaN,0.00,0,NaN,RPRPMLLJNHMFFCB87CC
3,臺中市,北區,房地(土地+建物)+車位,臺中市北區文昌東二街116巷1~30號,22.94,住,NaN,NaN,1060427,土地1建物1車位1,...,有,有,無,20000,113.0,坡道平面,41.14,0,NaN,RPWNMLLJNHMFFBB08CC
4,臺中市,梧棲區,土地,忠孝段451~480地號,3093.00,住,NaN,NaN,1060425,土地1建物0車位0,...,有,無,無,11500,4.0,NaN,0.00,0,NaN,RPVNMLLJNHMFFCL09CC
5,臺中市,西屯區,房地(土地+建物),臺中市西屯區河南路二段1~30號,25.20,住,NaN,NaN,1060425,土地1建物1車位0,...,有,無,有,12000,61.0,NaN,0.00,0,NaN,RPXOMLLJNHMFFCB48DC
6,臺中市,梧棲區,房地(土地+建物),臺中市梧棲區民生西街181~210號,115.23,住,NaN,NaN,1060427,土地1建物1車位0,...,有,無,無,15000,80.0,NaN,0.00,0,NaN,RPWNMLLJNHMFFCL66DC
7,臺中市,西區,房地(土地+建物),臺中市西區三民路一段50巷1~30號,2.73,住,NaN,NaN,1060405,土地2建物1車位0,...,有,有,有,4000,113.0,NaN,0.00,0,NaN,RPQNMLMJNHMFFAB47CC
8,臺中市,北屯區,房地(土地+建物)+車位,臺中市北屯區軍福七路151~180號,20.76,住,NaN,NaN,1060410,土地1建物1車位1,...,有,無,有,25000,137.0,坡道平面,27.05,0,NaN,RPRNMLMJNHMFFBB67CC
9,臺中市,南屯區,房地(土地+建物),臺中市南屯區大英街601~630號,289.26,住,NaN,NaN,1060418,土地1建物1車位0,...,無,無,無,60000,207.0,NaN,0.00,0,1.106年8月1日起至109年4月30日，每月租金陸萬元整。2.109年5月1日起至113...,RPORMLMJNHMFFCB87CC


In [31]:
dfs = [pd_read_lrv(f) for f in csv_files] 

In [26]:
# list(map(lambda x: x.shape, dfs))

In [32]:
df_all = reduce(lambda x, y: pd.concat([x, y], axis=0), dfs)

In [33]:
dfs
df_all.shape[1]

29

In [35]:
assert df_all.shape[1] == 29, 'Dimension error'

In [36]:
df_all.head()

,county,鄉鎮市區,交易標的,土地區段位置或建物區門牌,土地移轉總面積平方公尺,都市土地使用分區,非都市土地使用分區,非都市土地使用編定,交易年月日,交易筆棟數,...,建物現況格局-衛,建物現況格局-隔間,有無管理組織,總價元,單價每平方公尺,車位類別,車位移轉總面積平方公尺,車位總價元,備註,編號
0,臺北市,文山區,房地(土地+建物),臺北市文山區保儀路109巷23弄1~30號,14.50,住,NaN,NaN,1060208,土地1建物1車位0,...,1,有,無,7280000,82549.0,NaN,0.00,0,含增建或未登記建物。,RPSOMLMJNHMFFAA47CA
1,臺北市,中正區,房地(土地+建物),臺北市中正區中華路二段313巷1~30號,10.53,住,NaN,NaN,1060409,土地1建物1車位0,...,1,有,無,4000000,156067.0,NaN,0.00,0,NaN,RPPNMLPJNHMFFAA37CA
2,臺北市,文山區,車位,臺北市文山區興順街151~180號,4.60,住,NaN,NaN,1060320,土地0建物0車位1,...,0,有,有,1350000,NaN,坡道平面,36.13,1350000,NaN,RPRNMLTJNHMFFAA97CA
3,臺北市,文山區,房地(土地+建物),臺北市文山區景中街30巷1~30號,1.24,其他,NaN,NaN,1060415,土地1建物1車位0,...,0,無,有,1000000,110497.0,NaN,0.00,0,NaN,RPUSMLPKNHMFFAA48CA
4,臺北市,萬華區,房地(土地+建物),臺北市萬華區峨眉街91~120號,3.10,商,NaN,NaN,1060308,土地1建物1車位0,...,2,有,有,7250000,234780.0,NaN,0.00,0,含增建或未登記建物。,RPPPMLMJNHMFFBA37CA


In [39]:
df_all['county'].value_counts()

新北市    13549
臺中市    11329
桃園縣    10512
高雄市    10145
臺南市     6733
臺北市     5396
彰化縣     3193
屏東縣     2479
新竹縣     2456
雲林縣     2145
苗栗縣     2130
新竹市     1790
嘉義縣     1694
宜蘭縣     1649
南投縣     1628
基隆市     1239
花蓮縣     1095
嘉義市      896
臺東縣      749
金門縣      309
澎湖縣      281
連江縣       11
Name: county, dtype: int64

## Export

In [38]:
# df_all.to_csv('data/df_all_2017S3.csv', encoding='utf-8', index=False)

---

In [88]:
def pd_read_csv_cp950(file):
    try:
        df = pd.read_csv(file, encoding='cp950')
    except Exception as e:
        from io import StringIO
        with open(file , "rb") as  fin :
            text = fin.read()
            text = text.decode('cp950', errors='ignore')
            df = pd.read_csv(StringIO(text), error_bad_lines=False)
    return df

def pd_read_lrv(file):
    
    county_dict = dict(
        [('C', '基隆市'),
        ('A', '臺北市'),
        ('F', '新北市'),
        ('H', '桃園縣'),
        ('O', '新竹市'),
        ('J', '新竹縣'),
        ('K', '苗栗縣'),
        ('B', '臺中市'),
        ('M', '南投縣'),
        ('N', '彰化縣'),
        ('P', '雲林縣'),
        ('I', '嘉義市'),
        ('Q', '嘉義縣'),
        ('D', '臺南市'),
        ('E', '高雄市'),
        ('T', '屏東縣'),
        ('G', '宜蘭縣'),
        ('U', '花蓮縣'),
        ('V', '臺東縣'),
        ('X', '澎湖縣'),
        ('W', '金門縣'),
        ('Z', '連江縣')]
    )
    county_code = file.split('/')[-1][0]
    county = county_dict.get(county_code)
    df = pd_read_csv_cp950(file)
    df.insert(loc=0, column='county', value=county)
    return df

def read_and_combine(folder):

    print('Processing...', str(folder))
    dest_folder = Path(folder)

    csv_files = [f.as_posix() for f in dest_folder.glob('*.CSV') if re.search(r'land_[AB].CSV$', f.as_posix())]
    csv_files.sort()

    dfs = [pd_read_lrv(f) for f in csv_files]
    for df in dfs:
#         print(df.shape)
        if df.shape[1] != 29:
            print(df.head())
            raise Exception('Dimension error')
    
    df_all = reduce(lambda x, y: pd.concat([x, y], axis=0), dfs)

    # Export to csv
    df_all.to_csv(dest_folder.with_suffix('.csv').as_posix(), encoding='utf-8', index=False)
    
    print(df_all.shape)
    return df_all

In [91]:
read_and_combine('./data/2016S1/')
read_and_combine('./data/2016S2/')
read_and_combine('./data/2016S3/')
read_and_combine('./data/2016S4/')
read_and_combine('./data/2017S1/')

Processing... ./data/2016S1/


b'Skipping line 1420: expected 28 fields, saw 29\nSkipping line 1454: expected 28 fields, saw 29\nSkipping line 1455: expected 28 fields, saw 29\nSkipping line 1456: expected 28 fields, saw 29\nSkipping line 1457: expected 28 fields, saw 29\nSkipping line 1458: expected 28 fields, saw 29\nSkipping line 1512: expected 28 fields, saw 29\nSkipping line 1559: expected 28 fields, saw 29\nSkipping line 1560: expected 28 fields, saw 29\nSkipping line 1561: expected 28 fields, saw 29\nSkipping line 3251: expected 28 fields, saw 29\n'


(96946, 29)
Processing... ./data/2016S2/
(54881, 29)
Processing... ./data/2016S3/


b'Skipping line 1148: expected 28 fields, saw 29\n'


(76494, 29)
Processing... ./data/2016S4/
(70711, 29)
Processing... ./data/2017S1/
(78475, 29)
Processing... ./data/2017S2/


b'Skipping line 661: expected 28 fields, saw 29\nSkipping line 726: expected 28 fields, saw 29\nSkipping line 1883: expected 28 fields, saw 29\nSkipping line 1884: expected 28 fields, saw 29\n'


(66949, 29)


,county,鄉鎮市區,交易標的,土地區段位置或建物區門牌,土地移轉總面積平方公尺,都市土地使用分區,非都市土地使用分區,非都市土地使用編定,交易年月日,交易筆棟數,...,建物現況格局-衛,建物現況格局-隔間,有無管理組織,總價元,單價每平方公尺,車位類別,車位移轉總面積平方公尺,車位總價元,備註,編號
0,臺北市,中正區,房地(土地+建物),臺北市中正區金門街1~30號,10.71,住,NaN,NaN,1051102,土地1建物1車位0,...,0,無,無,7710000,241996.0,NaN,0.00,0,二親等買賣,RPQNMLRJKHMFFAA37CA
1,臺北市,文山區,土地,政大段二小段91~120地號,104.00,其他,NaN,NaN,1051208,土地1建物0車位0,...,0,有,無,11300000,108654.0,NaN,0.00,0,親友、員工或其他特殊關係間之交易。含增建或未登記建物。,RPUNMLMJKHMFFAA27CA
2,臺北市,中正區,房地(土地+建物),臺北市中正區金華街1~30號,12.88,商,NaN,NaN,1051229,土地5建物1車位0,...,2,有,有,29680000,236701.0,NaN,0.00,0,含增建或未登記建物。,RPPRMLTJKHMFFAA87CA
3,臺北市,中正區,車位,臺北市中正區羅斯福路二段8巷1~30號,0.15,住,NaN,NaN,1060116,土地0建物0車位1,...,0,有,有,1300000,NaN,坡道機械,15.84,1300000,NaN,RPSOMLPKKHMFFAA57CA
4,臺北市,中正區,房地(土地+建物),臺北市中正區新生南路一段1~30號,15.23,其他,NaN,NaN,1051221,土地2建物1車位0,...,2,有,有,27500000,227104.0,NaN,0.00,0,含增建或未登記建物。,RPSNMLQJKHMFFBA57CA
5,臺北市,萬華區,房地(土地+建物),臺北市萬華區長沙街二段31~60號,17.00,商,NaN,NaN,1051211,土地2建物1車位0,...,1,有,無,22080000,140530.0,NaN,0.00,0,含增建或未登記建物。,RPTNMLRJKHMFFBA77CA
6,臺北市,中正區,房地(土地+建物)+車位,臺北市中正區中華路一段95巷1~30號,20.92,商,NaN,NaN,1051230,土地1建物1車位2,...,0,無,無,78500000,302036.0,坡道平面,91.27,6700000,NaN,RPSNMLPKKHMFFBA57CA
7,臺北市,中山區,房地(土地+建物),臺北市中山區新生北路二段58巷1~30號,10.84,商,NaN,NaN,1050410,土地1建物1車位0,...,1,有,有,10600000,138635.0,NaN,0.00,0,共有人數2人。含增建或未登記建物。,RPSPMLQJKHMFFCA87CA
8,臺北市,內湖區,房地(土地+建物),臺北市內湖區康樂街151巷1~30號,18.41,住,NaN,NaN,1051217,土地1建物1車位0,...,1,有,有,10400000,157887.0,NaN,0.00,0,價格含增建或未辦保存登記建物,RPQOMLQJKHMFFCA57CA
9,臺北市,中山區,房地(土地+建物),臺北市中山區民權東路二段92巷1弄1~30號,12.32,住,NaN,NaN,1051221,土地1建物1車位0,...,1,有,無,7300000,172863.0,NaN,0.00,0,NaN,RPORMLRJKHMFFCA77CA


In [92]:
read_and_combine('./data/2017S2/')

Processing... ./data/2017S2/


b'Skipping line 661: expected 28 fields, saw 29\nSkipping line 726: expected 28 fields, saw 29\nSkipping line 1883: expected 28 fields, saw 29\nSkipping line 1884: expected 28 fields, saw 29\n'


(66949, 29)


,county,鄉鎮市區,交易標的,土地區段位置或建物區門牌,土地移轉總面積平方公尺,都市土地使用分區,非都市土地使用分區,非都市土地使用編定,交易年月日,交易筆棟數,...,建物現況格局-衛,建物現況格局-隔間,有無管理組織,總價元,單價每平方公尺,車位類別,車位移轉總面積平方公尺,車位總價元,備註,編號
0,臺北市,中正區,房地(土地+建物),臺北市中正區金門街1~30號,10.71,住,NaN,NaN,1051102,土地1建物1車位0,...,0,無,無,7710000,241996.0,NaN,0.00,0,二親等買賣,RPQNMLRJKHMFFAA37CA
1,臺北市,文山區,土地,政大段二小段91~120地號,104.00,其他,NaN,NaN,1051208,土地1建物0車位0,...,0,有,無,11300000,108654.0,NaN,0.00,0,親友、員工或其他特殊關係間之交易。含增建或未登記建物。,RPUNMLMJKHMFFAA27CA
2,臺北市,中正區,房地(土地+建物),臺北市中正區金華街1~30號,12.88,商,NaN,NaN,1051229,土地5建物1車位0,...,2,有,有,29680000,236701.0,NaN,0.00,0,含增建或未登記建物。,RPPRMLTJKHMFFAA87CA
3,臺北市,中正區,車位,臺北市中正區羅斯福路二段8巷1~30號,0.15,住,NaN,NaN,1060116,土地0建物0車位1,...,0,有,有,1300000,NaN,坡道機械,15.84,1300000,NaN,RPSOMLPKKHMFFAA57CA
4,臺北市,中正區,房地(土地+建物),臺北市中正區新生南路一段1~30號,15.23,其他,NaN,NaN,1051221,土地2建物1車位0,...,2,有,有,27500000,227104.0,NaN,0.00,0,含增建或未登記建物。,RPSNMLQJKHMFFBA57CA
5,臺北市,萬華區,房地(土地+建物),臺北市萬華區長沙街二段31~60號,17.00,商,NaN,NaN,1051211,土地2建物1車位0,...,1,有,無,22080000,140530.0,NaN,0.00,0,含增建或未登記建物。,RPTNMLRJKHMFFBA77CA
6,臺北市,中正區,房地(土地+建物)+車位,臺北市中正區中華路一段95巷1~30號,20.92,商,NaN,NaN,1051230,土地1建物1車位2,...,0,無,無,78500000,302036.0,坡道平面,91.27,6700000,NaN,RPSNMLPKKHMFFBA57CA
7,臺北市,中山區,房地(土地+建物),臺北市中山區新生北路二段58巷1~30號,10.84,商,NaN,NaN,1050410,土地1建物1車位0,...,1,有,有,10600000,138635.0,NaN,0.00,0,共有人數2人。含增建或未登記建物。,RPSPMLQJKHMFFCA87CA
8,臺北市,內湖區,房地(土地+建物),臺北市內湖區康樂街151巷1~30號,18.41,住,NaN,NaN,1051217,土地1建物1車位0,...,1,有,有,10400000,157887.0,NaN,0.00,0,價格含增建或未辦保存登記建物,RPQOMLQJKHMFFCA57CA
9,臺北市,中山區,房地(土地+建物),臺北市中山區民權東路二段92巷1弄1~30號,12.32,住,NaN,NaN,1051221,土地1建物1車位0,...,1,有,無,7300000,172863.0,NaN,0.00,0,NaN,RPORMLRJKHMFFCA77CA
